In [2]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

from svgpathtools import parse_path

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### Setup

In [7]:
# directory & file hierarchy
proj_dir = os.path.abspath('../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)  
    
## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))        
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

<module 'analysis_helpers' from '/Users/judithfan/semantic_parts/analysis/analysis_helpers.pyc'>

#### set up connection to mongo

In [ ]:
#### set vars 
import pandas as pd
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['svgAnnotation']
coll = db['examples']

# which iteration name should we use?
iterationName = 'testing9'

### get some descriptive stats on our annotation data!

In [ ]:
num_annotations = coll.find({'iterationName':iterationName}).count()
print 'We have {} annotations so far.'.format(num_annotations)

#### build up annotation data csv

In [ ]:
## TODO remember to ignore researcher worker ID's in our dataset
jefan = ['A1MMCS8S8CTWKU','A1MMCS8S8CTWKV','A1MMCS8S8CTWKS']
hawkrobe = ['A1BOIDKD33QSDK']
megsano = ['A1DVQQLVZR7W6I']
kmukherjee = ['A1WU4IHJNQGVAY']
researchers = jefan + hawkrobe + megsano + kmukherjee

In [ ]:
## TODO use a helper function to get complete and valid HITs 

In [ ]:
## categories that are in play
main_category = 'chair'
dataset = 'chairs4'

In [ ]:
### get all the sketch recs!
sketch_recs = coll.find({'iterationName':iterationName}).sort('time')

### initialize a bunch of stuff
orig_gameID = [] # the gameID from which this sketch was sourced
category = [] # e.g., "chair"
target = [] # e.g., "inlay"
condition = [] # e.g., "closer" vs. "further" or "repeated" vs. "control
trial_num = []
time_submitted = [] # when the participant clicked "next sketch"
time_labeled = [] # unique to each spline labeled
time_clicked = [] # when this spline was clicked/selected
num_strokes_in_sketch = [] # how many strokes in this sketch
num_splines_in_sketch = [] # how many spline elements in this sketch
stroke_num = [] # which stroke number this labeled spline came from
cumulative_spline_num = [] # spline index in the cumulative spline sequence for the entire sketch
within_stroke_spline_num = [] # spline index for the current stroke
label = [] # the label provided by the participant
spline_svg_string = [] # the svg spline string that earned this label
sketch_svg_string = [] # the entire svg string correponding to this sketch

for sketch_ind,sketch in enumerate(sketch_recs):
    print 'Analyzing {} of {} sketches ...'.format(sketch_ind,sketch_recs.count())
    ## get annotations embedded within record
    annotations_string = sketch['annotations']  
    ## convert to json dictionary
    _annotations_dict = json.loads(annotations_string)   
    annotations_dict = _annotations_dict[0][main_category]
    num_splines = len(annotations_dict)
    for annotation in annotations_dict:
        ## get spline-level metadata
        label.append(annotation['label'])
        stroke_num.append(annotation['strokeNum'])
        spline_svg_string.append(annotation['svgString'])
        cumulative_spline_num.append(annotation['cumulativeSplineNum'])
        within_stroke_spline_num.append(annotation['withinStrokeSplineNum'])
        time_clicked.append(annotation['timeClicked'])
        time_labeled.append(annotation['timeLabeled'])
        ## get sketch-level metadata
        orig_gameID.append(sketch['originalGameID'])        
        category.append(sketch['category'])
        target.append(sketch['target'])
        condition.append(sketch['condition'])
        time_submitted.append(sketch['time'])
        trial_num.append(sketch['trialNum'])
#         assert sketch['numSplines']==num_splines
        num_splines_in_sketch.append(sketch['numSplines'])
        num_strokes_in_sketch.append(sketch['numStrokes'])
        sketch_svg_string.append(sketch['svg'])
                                    

In [ ]:
## make group data csv 

D = pd.DataFrame([orig_gameID, category, target, condition, trial_num, time_submitted,\
                 time_labeled, time_clicked, num_strokes_in_sketch, num_splines_in_sketch,\
                 stroke_num, cumulative_spline_num, within_stroke_spline_num, label,\
                 spline_svg_string, sketch_svg_string])
D = D.transpose()
D.columns = ['orig_gameID', 'category', 'target', 'condition', 'trial_num', 'time_submitted',\
             'time_labeled', 'time_clicked', 'num_strokes_in_sketch', 'num_splines_in_sketch',\
             'stroke_num', 'cumulative_spline_num', 'within_stroke_spline_num', 'label',\
             'spline_svg_string', 'sketch_svg_string']

print 'Annotations dataframe contains {} rows and {} columns.'.format(D.shape[0],D.shape[1])

In [ ]:
## save out csv to results dir
D.to_csv(os.path.join(results_dir,'svg_annotations_{}.csv'.format(dataset)))

#### get descriptive stats on annotation data

In [ ]:
## Some starting questions about sketchpad_basic dataset

## Annotation task
## How consistent are annotators for the same object? Inter-rater reliability

## Content
## What does the part distribution look like for each object? Presence/absence.
## How does the part distribution differ between contexts, matched by object?

## Style
## How much arc length is spent on each part 

## Dynamics

## 2. Is the entropy of the part distribution different for close trials than far trials?
## Here we might use arc length as a measure of how much was expended for each part.